In [1]:
import logging
import os
import numpy as np
import pandas as pd
import time


from data_loader import DataLoader

In [2]:
if __name__ == '__main__':

    path = 'C:/Users/Akis/OneDrive/Masters/Web Economics/Data/'

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    logging.info('starting train_via_rf.py')

    target_col_name = 'click'
    data_filename = 'validation.csv'

    logging.info('loading DF ' + data_filename)
    train_dl = DataLoader()
    train_dl.load_file(path, data_filename)
    df = train_dl.get_df_copy()

2018-02-27 21:55:59,753 : INFO : starting train_via_rf.py
2018-02-27 21:55:59,757 : INFO : loading DF validation.csv
2018-02-27 21:55:59,758 : INFO : Loading C:/Users/Akis/OneDrive/Masters/Web Economics/Data/validation.csv
2018-02-27 21:56:01,453 : INFO : file  loaded


In [21]:
class Scorer(object):

    def second_largest(self,numbers):
        count = 0
        topBid = secondBid = -1.0
        for x in numbers:
            count += 1
            if x > secondBid:
                if x >= topBid:
                    topBid, secondBid = x, topBid
                else:
                    secondBid = x
        return secondBid if count >= 2 else None
    
    def FormulaCombineCTRBaseBid(self,CTR,BaseBid):
        CTR=(((CTR+0.5)**2)-0.5)
        #CTR=max(0.1,CTR).all()
        prices = (CTR/0.5)*BaseBid
        return prices

    def set_df(self,clicksDF,predictionsBaseBid,predictionsCTR,column_names, budget, reserve):
        

        CTRdf=clicksDF.copy(True)
        BaseBiddf=clicksDF.copy(True)
        for i in range(0,len(predictionsCTR),1):
            name=column_names[i]
            CTRdf[name] = predictionsCTR[i]
            BaseBiddf[name] = predictionsBaseBid[i]
        
        #df = df.copy(True)
        balances = np.ones(len(column_names)) * budget
        df_teams_CTR = CTRdf[column_names]
        df_teams_BaseBid = BaseBiddf[column_names]
        CTRdf['winner'] = -1
        CTRdf['price'] = 0
        logging.info('calculating price paid (second highest). This takes some time...')
           
        for i in range(len(CTRdf)):
            #prices = (df_teams_CTR.loc[i].values/0.5)*df_teams_BaseBid.loc[i].values
            prices = self.FormulaCombineCTRBaseBid(df_teams_CTR.loc[i].values,df_teams_BaseBid.loc[i].values)
            #print(prices)
            valid1 = balances > prices
            valid2 = prices > 0.0
            valid3 = valid1 * valid2
            new_prices = valid3 * prices
            winner = np.argmax(new_prices)
            if (new_prices[winner]>0):
                price = self.second_largest(new_prices)
                price = np.max([reserve,price])
                CTRdf.loc[i, 'price'] = price
                CTRdf.loc[i, 'winner'] = winner
                balances[winner] -= price

            #if i % int(len(CTRdf)/49) == 0:
            #    logging.info('%.2f complete',(float(i)/len(CTRdf)))

            if np.sum(balances > 0.0) == 0: # if no balances are positive, we don't need to process the rest of the data frame
                break

        logging.info('calculating wins by team')
        wins_by_team = CTRdf.winner.value_counts()
        teams = CTRdf.winner.unique()

        team_names = []
        team_wins = []
        team_clicks = []
        for team_index in teams:
            if team_index >= 0:
                temp_df = CTRdf.loc[CTRdf['winner'] == team_index]
                clicks = np.sum(temp_df['click'].values)
                team_name = df_teams_CTR.columns[team_index]
                team_names.append(team_name)
                team_wins.append(wins_by_team[team_index])
                team_clicks.append(clicks)

                logging.info(
                    'team ' + team_name + ' ' + str(team_index) + ' wins:' + str(wins_by_team[team_index]) + ' clicks:' + str(clicks))
        df_summary = pd.DataFrame({"team_name": team_names, "win": team_wins, "click": team_clicks})
        return(df_summary,CTRdf)

In [4]:
clicksFile = df[['click']].copy(True)
Random1BaseBid = np.random.randint(227, 230, clicksFile.shape[0])
Constant1BaseBid = np.ones(clicksFile.shape[0]) * 278
Random2BaseBid = np.random.randint(297, 301, clicksFile.shape[0])
Random1CTR = np.random.randint(0, 100, clicksFile.shape[0])/100
Constant1CTR = np.random.randint(0, 100, clicksFile.shape[0])/100
Random2CTR = np.random.randint(0, 100, clicksFile.shape[0])/100
print(Constant1BaseBid)
print(Constant1CTR)
#print(clicksFile.head())


[278. 278. 278. ... 278. 278. 278.]
[0.53 0.15 0.17 ... 0.82 0.14 0.78]


In [5]:
#for i in range(1,300,10):
#    print((Random2CTR[i]/0.5)*Random2BaseBid[i])

In [22]:
start = time.time()
s = Scorer()
df_summary, df_full = s.set_df(clicksFile,[Random1BaseBid,Random2BaseBid,Constant1BaseBid],[Random1CTR,Random2CTR,Constant1CTR],['bidTeam1','bidTeam2','bidTeam3'],625000, 227)

end = time.time()
print("Time:",end - start,"s")

print(df_summary.head())

2018-02-27 21:59:57,700 : INFO : calculating price paid (second highest). This takes some time...


303925


2018-02-27 22:00:28,991 : INFO : calculating wins by team
2018-02-27 22:00:28,998 : INFO : team bidTeam2 1 wins:1668 clicks:4
2018-02-27 22:00:29,000 : INFO : team bidTeam1 0 wins:2272 clicks:3
2018-02-27 22:00:29,002 : INFO : team bidTeam3 2 wins:1772 clicks:0


Time: 31.333794832229614 s
   click team_name   win
0      4  bidTeam2  1668
1      3  bidTeam1  2272
2      0  bidTeam3  1772


In [18]:
CTR=max(0.1,0.1)
print(CTR)

0.1
